In [3]:
import numpy as np
import pandas as pd

# preprocessing
import os
from datetime import datetime

# EDA
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats
from scipy.stats import skew
from scipy.stats import spearmanr

# Learning algorithms
import sklearn
from sklearn.linear_model import *
from sklearn.svm import SVR
from sklearn.cluster import KMeans

import lightgbm as lgb
from lightgbm import LGBMRegressor
# import catboost

# model validation
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

SEED = 2
np.random.seed(SEED)

In [4]:
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False 

#f_path = "c:/Windows/Fonts/AppleGothic.ttf"(맥)
f_path = "c:/Windows/Fonts/malgun.ttf"

font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font',family = font_name)

In [5]:
main_path = 'D:/dacon/data/'
data_dir = 'energy'
dir_path = f'{main_path}/{data_dir}'

test_name = 'test.csv'
train_name = 'train.csv'
building_name = 'building_info.csv'
sample_submission_name = 'sample_submission.csv'

In [6]:
def rename_dict(df):
    # 이름 정리
    targets = df.columns[df.columns.str.contains('\(')]
    if len(targets) == 7:
        # train_df
        df.rename(columns = {
            '기온(C)':'기온',
            '강수량(mm)':'강수량',
            '풍속(m/s)':'풍속',
            '습도(%)':'습도',
            '일조(hr)':'일조',
            '일사(MJ/m2)':'일사',
            '전력소비량(kWh)':'전력소비량'
        },inplace = True)
    elif len(targets) == 4:
        # test_df
        df.rename(columns = {
            '기온(C)':'기온',
            '강수량(mm)':'강수량',
            '풍속(m/s)':'풍속',
            '습도(%)':'습도',
        },inplace = True)
    else:
        # building_df
        df.rename(columns = {
            '연면적(m2)':'연면적',
            '냉방면적(m2)':'냉방면적', 
            '태양광용량(kW)':'태양광용량',
            'ESS저장용량(kWh)':'ESS저장용량',
            'PCS용량(kW)':'PCS용량'
        },inplace = True)
        
    return df

def data_datetime_setting(df):
    date_format = "%Y%m%d %H"
    # 일시 컬럼 이용하여 요일 확인
    df['요일'] = df['일시'].apply(lambda x: datetime.strptime(x, date_format).strftime("%a"))
    # 일시 컬럼 이용하여 연/월/일/시간 컬럼추가
    df['일시'] = pd.to_datetime(df['일시'], format='%Y%m%d %H')
    df['연'] = df['일시'].dt.year
    df['월'] = df['일시'].dt.month
    df['일'] = df['일시'].dt.day
    df['시간'] = df['일시'].dt.hour
    return df

def data_preprocessing(df):
    try:
        df['강수량'].fillna(0, inplace=True)
        df['풍속'].fillna(method='ffill', inplace=True)
        df['습도'].fillna(method='ffill', inplace=True)
        df['일조'].fillna(0, inplace=True)
        df['일사'].fillna(0, inplace=True)
    except:
        df['강수량'].fillna(0, inplace=True)
        df['일조'] = 0
        df['일사'] = 0
    return df

def merge_dfs(df):
    merge_df = df.merge(building_df)
    for col in merge_df.columns:
        merge_df[col] = merge_df[col].replace('-',0)
        merge_df[col] = merge_df[col].fillna(0)

    merge_df['태양광용량'] = merge_df['태양광용량'].astype('float64')
    merge_df['ESS저장용량'] = merge_df['ESS저장용량'].astype('float64')
    merge_df['PCS용량'] = merge_df['PCS용량'].astype('float64')
    merge_df.drop(['num_date_time'],axis=1,inplace=True) # 둘 다 없어도 알 수 있는 데이터라서
    return merge_df

# 요일, 건물유형은 원핫인코딩하기 > pd.get_dummies()


In [7]:
os.listdir(dir_path)

['building_info.csv', 'sample_submission.csv', 'test.csv', 'train.csv']

In [8]:
# 제출 형식
pd.read_csv(f'{dir_path}/{sample_submission_name}').columns

Index(['num_date_time', 'answer'], dtype='object')

In [9]:
test_df = pd.read_csv(f'{dir_path}/{test_name}')
train_df = pd.read_csv(f'{dir_path}/{train_name}')
building_df = pd.read_csv(f'{dir_path}/{building_name}')

In [10]:
train_df = rename_dict(df = train_df)
# train_df = data_datetime_setting(df=train_df)
train_df = data_preprocessing(df=train_df)

building_df = rename_dict(df = building_df)

test_df = rename_dict(df = test_df)
test_df = data_preprocessing(df=test_df)

In [11]:
train_merge = merge_dfs(df = train_df)
test_merge = merge_dfs(df = test_df)

In [12]:
from tqdm import tqdm
import pandas as pd
import statsmodels.api as sm
predicts = []
# 데이터 불러오기 (예시)
for n in tqdm(range(1,len(train_merge['건물번호'].unique())+1)):
    train__ = train_merge[train_merge['건물번호'] == n]
    test__ = test_merge[test_merge['건물번호'] == n]
    n_data = train__.copy()
    t_data = test__.copy()
    n_data.set_index('일시', inplace=True)
    t_data.set_index('일시', inplace=True)

    # 다중 회귀를 위한 피쳐 선택
    features = ['건물번호', '기온', '강수량', '풍속', '습도', '일조', '일사', '연면적',
       '냉방면적', '태양광용량', 'ESS저장용량', 'PCS용량']
    # 종속 변수 설정
    target = '전력소비량'

#     # 훈련 데이터 분할 (예시)
#     index = int(len(data)*4/5)
#     train_data = data.iloc[:index]
#     test_data = data.iloc[index:]

    # 피쳐와 종속 변수 설정
    X_train = n_data[features]
    y_train = n_data[target]
    X_test = t_data[features]


    # 상수(intercept)를 추가하여 회귀 모델 피팅
    X_train = sm.add_constant(X_train)
    model = sm.OLS(y_train, X_train).fit()
    # 테스트 데이터 예측
    X_test = sm.add_constant(X_test)
    predictions = model.predict(X_test)
    
    test_merge.loc[test_merge[test_merge['건물번호']==n].index,'answer'] = predicts[n-1].values
    
    # 모델 결과 출력
    print(model.summary())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


IndexError: list index out of range

건물별로 

In [23]:
test_merge['num_date_time'] = test_merge['건물번호'].astype('str') + '_' + test_merge['일시'].astype('str')

In [26]:
test_merge[['num_date_time','answer']].to_csv('../output/submission.csv',index=False)

In [27]:
test_merge[['num_date_time','answer']]

,num_date_time,answer
0,1_20220825 00,1816.315399
1,1_20220825 01,1569.815399
2,1_20220825 02,1675.815399
3,1_20220825 03,1630.815399
4,1_20220825 04,1535.315399
...,...,...
16795,100_20220831 19,662.838498
16796,100_20220831 20,555.760373
16797,100_20220831 21,530.842404
16798,100_20220831 22,552.637326
